<a href="https://colab.research.google.com/gist/jcheong0428/c16146b386ea60fab888b56e8e5ee747/openface_shared.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *Two-Ts*: Reprodução de Expressões Faciais em Cabeça Robótica com Tecnologia 3D

Trabalho de Mestrado em Ciência da Computação da Universidade Federal do ABC (UFABC).

Discente: Tamires dos Santos.

Orientador: Wagner Tanaka Botelho.

Bolsas Recebidas:
+ Treinamento Técnico (TT-3) do Programa PIPE da Fundação de Amparo à Pesquisa do Estado de São Paulo - FAPESP, com o processo número 2019/08079-1, pela  Startup NTU Software Technology do Projeto PIPE Fase 2 (Processo Número: 2018/04306-0) no período de Junho/2019 a Maio/2020;
+ Bolsa de mestrado da Coordenação de Aperfeiçoamento de Pessoal de Nível Superior (CAPES) no período de Outubro/2020 a Fevereiro/2021.



Data de modificação: Julho/2021.

# Sistema de validação das Expressões da *Two-Ts*


Este notebook usa o projeto de código aberto [OpenFace](https://github.com/TadasBaltrusaitis/OpenFace), de Tadas Baltrusaitis, para detectar a partir de uma face, as Unidades de Ações (UAs) baseadas no FACS criada por [Paul Ekman](https://www.paulekman.com/) em  parceria com Wallace Friesen. 

Além da instruções presentes no github do OpenFace, também foi utilizado como base o artigo e notebbok criado por Jin Hyun Cheong (https://towardsdatascience.com/how-to-extract-facial-expressions-head-pose-and-gaze-from-any-youtube-video-2aa6590c2bb6).

O objetivo do código é realizar a validação das expressões faciais, durante a reprodução das seis emoções básicas e universais, na face da *Two-Ts* virtual e real, em comparação com a da discente Tamires dos Santos. 

### Instalação do OpenFace

In [ ]:
import os
from os.path import exists, join, basename, splitext

################# Install CUDA 10.0 #################
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update
!wget  --no-clobber https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
!sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
!apt-get update
!apt-get install cuda-10-0
!wget --no-clobber http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64/nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb
!apt install ./nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb
!apt-get update
#######################################################

################# Install OpenFace #################
git_repo_url = 'https://github.com/TadasBaltrusaitis/OpenFace.git'
project_name = splitext(basename(git_repo_url))[0]
!git clone -q --depth 1 $git_repo_url
#######################################################

################# Install other packages #################
!wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
!tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local

!sudo apt-get update
!sudo apt-get install build-essential 
!sudo apt-get install g++-8
#######################################################

# Finishing the installation
!cd OpenFace && bash ./download_models.sh && sudo bash ./install.sh


### Selecionar imagens 

Localiza as imagens e extrae os dados.

In [ ]:
# Create folder to store data
!rm -rf processed

!./OpenFace/build/bin/FaceLandmarkImg -f virtual.jpg -out_dir processed -of dataImage1   #Two-Ts virtual
!./OpenFace/build/bin/FaceLandmarkImg -f real.jpg -out_dir processed -of dataImage2  #Two-Ts real
!./OpenFace/build/bin/FaceLandmarkImg -f tamires.png -out_dir processed -of dataImage3  #Discente Tamires

###Carregar os dados de cada imagem

Será carregado os dados das imagens em Pandas, renomeando as colunas para eliminar espaços vazios.

In [ ]:
import pandas as pd, seaborn as sns
import matplotlib.pyplot as plt 

df1 = pd.read_csv('processed/dataImage1.csv')
df1.columns = [col.replace(" ", "") for col in df1.columns]

df2 = pd.read_csv('processed/dataImage2.csv')
df2.columns = [col.replace(" ", "") for col in df2.columns]

df3 = pd.read_csv('processed/dataImage3.csv')
df3.columns = [col.replace(" ", "") for col in df3.columns]

print(df1)
print(df2)
print(df3)

### Plotar os resultados das UAs


In [ ]:
import re
import numpy as np

# Function to read the column values of the AUs
def valuesAUs(id_image):
  au_values = []
  au_list = ['01', '02', '04', '05', '06' ,'07', '09', '10', '12', '14', '15', '17', '20', '23', '25', '26' , '45']
  
  for i in range(0, len(au_list)):
    x = 'AU'+ au_list[i] +"_r"
    if id_image == 1:
        au_values.append(df1[x][0])
    if id_image == 2:
      au_values.append(df2[x][0])
    if id_image == 3:
      au_values.append(df3[x][0])
  
  return au_values

listName = ['UA1', 'UA2', 'UA4', 'UA5', 'UA6' ,'UA7', 'UA9', 'UA10', 'UA12', 'UA14', 'UA15', 'UA17', 'UA20', 'UA23', 'UA25', 'UA26' , 'UA45']

#au_regex_pat = re.compile(r'^AU[0-9]+_r$')
#au_columns1 = df1.columns[df1.columns.str.contains(au_regex_pat)]

au_values1 = valuesAUs(1)
au_values2 = valuesAUs(2)
au_values3 = valuesAUs(3)

image1 =  np.arange(len(au_values1))
image2 = [i + 0.2 for i in image1]
image3 = [i + 0.4 for i in image1]

plt.bar(image1, au_values1, width=0.2, label = 'Two-Ts Virtual', edgecolor ='g', color = 'white')
plt.bar(image2, au_values2, width=0.2, label = 'Two-Ts Real', color = 'gray')
plt.bar(image3, au_values3, width=0.2, label = 'Tamires', color = 'black')
plt.xticks([i for i in range(len(au_values1))], listName)
plt.xticks(rotation=20)
plt.legend()
plt.gcf().set_size_inches(20, 5) 
plt.ylabel('Intensidade')
plt.xlabel('Unidade de Ação')
plt.show()